In [88]:
# Example content of the namespace.yml file - either run the command above or this one to create the namespace
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Namespace
metadata:
  name: promtest
EOF

namespace/promtest unchanged


In [93]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: ilox
  namespace: promtest
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: ilox
  ports:
    - port: 9091               # The Port of that the Prometheus connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the Prometheus connector if you want/need to debug it. 
EOF

service/ilox created


In [168]:
kubectl apply -f ipromtst.yml

configmap/ilopromtest-xml configured


In [169]:
kubectl -n promtest describe configmap/ilopromtest-xml

Name:         ilopromtest-xml
Namespace:    promtest
Labels:       <none>
Annotations:  <none>

Data
====
iloprometheus.key:
----
LVkjkV5j40Fl6U0O7ratU0RDf5W1NmAE0oqoQ2DaE74=
iloprometheus.xml:
----
<data> 
  <username>Prometheus</username>
  <user>gAAAAABig6ZJgST83R7v5AFjsOKiYt0bwqBdledJpF3-TnUBR5871eNttG8P-O_eYA55-5NyyZgmmeWRnWy3hPAbUFk18_sWNw==</user>
  <password>gAAAAABig6ZJAZL8wS41b0snZ76VWo17JysUq0-k5VjnrNUCU1y5AAPw5p1MphkFcB_u6z0TgcPbmR9yJcX39Cx0Egwv5Lou8A==</password>
  <monitoringintervall>30</monitoringintervall>
  <logfile>iloprom2.log</logfile>
  <port>9091</port>      
  <server> <ILO_ip>10.1.24.64</ILO_ip>,<Rack>C5</Rack>,<Loc>20</Loc> </server>
  <server> <ILO_ip>10.1.71.235</ILO_ip>,<Rack>F5</Rack>,<Loc>1</Loc> </server>      
  <server> <ILO_ip>10.1.83.51</ILO_ip>,<Rack>F5</Rack>,<Loc>9</Loc> </server>
</data>

BinaryData
====

Events:  <none>


In [171]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ilox
  namespace: promtest
  labels:
    app: ilox
spec:
  selector:
    matchLabels:
      app: ilox
  template:
    metadata:
      labels:
        app: ilox
    spec:
      containers:
        - name: ilox
          image: tb1378/iloprometheus
          command: ["/usr/bin/python3"]
          args: ["/opt/prometheus/iloPromConnector.v3.0.py"]
          #command: ["/bin/sh"]
          #args: ["-c", "while true; do echo hello; sleep 10;done"]
          volumeMounts:
            - name: iloprometheusxml
              mountPath: /opt/prometheus/data
      volumes:
        - name: iloprometheusxml
          configMap:
            name: ilopromtest-xml   # the correct name of the configmap needs to be added here. 
EOF

deployment.apps/ilox created


In [172]:
kubectl -n promtest get pods -o wide
kubectl -n promtest get svc -o wide
kubectl -n promtest get deployments -o wide

NAME                    READY   STATUS    RESTARTS   AGE   IP             NODE                    NOMINATED NODE   READINESS GATES
ilox-6fc848bdd6-874p2   1/1     Running   0          3s    10.244.5.139   suo04k8sw3.demo.local   <none>           <none>
NAME   TYPE       CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE    SELECTOR
ilox   NodePort   10.104.227.154   <none>        9091:30336/TCP   3d3h   app=ilox
NAME   READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                 SELECTOR
ilox   1/1     1            1           4s    ilox         tb1378/iloprometheus   app=ilox


In [136]:
kubectl -n promtest logs pod/ilox-6fc848bdd6-fnbd4

In [173]:
kubectl -n promtest delete deployment/ilox

deployment.apps "ilox" deleted


In [40]:
kubectl -n promtest delete svc/ilox

service "ilo3" deleted
